In [1]:
import argparse
import gc
import numpy as np
import logging
import pickle
from datetime import datetime
from os import makedirs
from os.path import join
from pathlib import Path

import yaml
from scipy.sparse import load_npz
import scipy
from sklearn.decomposition import TruncatedSVD

seed = 42
SAVE_DIR = "/scratch/st-jiaruid-1/shenoy/svd-comp/"

In [2]:
paths = {
    'multiome': 
    {
      'x': '/arc/project/st-jiaruid-1/shenoy/data/multiome-sparse-rc/train_multi_inputs_raw_values.sparse.npz',
      'y': '/arc/project/st-jiaruid-1/shenoy/data/multiome-sparse-rc/train_multi_targets_raw_values.sparse.npz',
      'x_test': '/arc/project/st-jiaruid-1/shenoy/data/multiome-sparse-rc/test_multi_inputs_raw_values.sparse.npz'
    },
    'cite':
    {
      'x': '/arc/project/st-jiaruid-1/shenoy/data/multiome-sparse-rc/train_cite_inputs_raw_values.sparse.npz',
      'y': '/arc/project/st-jiaruid-1/shenoy/data/multiome-sparse-rc/train_cite_targets_raw_values.sparse.npz',
      'x_test': '/arc/project/st-jiaruid-1/shenoy/data/multiome-sparse-rc/test_cite_inputs_raw_values.sparse.npz'
    }
}

In [17]:
modality = 'multiome'

In [43]:
# Load Data
x = load_npz(paths[modality]["x"])

### Only keep `indices`

In [41]:
# threshold = 0.2
# indices = [i for i, x in enumerate(np.squeeze(np.std(x.toarray(), axis=0))) if x > threshold]

In [42]:
# len(indices)

#### Perform PCA

In [44]:
comp = 128

In [ ]:
# # perform preprocessing
# transform x and x_test
pca_x = TruncatedSVD(
    n_components=comp,
    random_state=seed,
)

x_transformed = pca_x.fit_transform(scipy.sparse.vstack([
    x, 
    load_npz(paths[modality]["x_test"])]
)[:, :])

x_train_transformed = x_transformed[: x.shape[0], :]
x_test_transformed = x_transformed[x.shape[0] :, :]    
del x, x_transformed
gc.collect()

In [ ]:
# save the processed arrays
input_dim = comp
input_type = modality
pickle.dump(
    x_train_transformed,
    open(join(SAVE_DIR, f"train_raw_input_{input_type}_{input_dim}_mod.pkl"), "wb"),
)
pickle.dump(
    x_test_transformed,
    open(join(SAVE_DIR, f"test_raw_input_{input_type}_svd{input_dim}_mod.pkl"), "wb"),
)